In [ ]:
from collections import defaultdict
import json

train_path = '/opt/ml/detection/dataset/train.json'
test_path = '/opt/ml/detection/dataset/test.json'
best_predictions_path = '/opt/ml/yolov5/runs/val/exp10/best_predictions.json' # exp를 변경하면서 돌림

score_thres = 0.5 # 이것 변경하면서 돌림

with open (train_path,'r') as f:
    train = json.load(f)
    
with open(test_path,'r') as f:
    test = json.load(f)
    
with open(best_predictions_path,'r') as f:
    best = json.load(f)
    preds = defaultdict(list)
    for i in best:
        preds[i['image_id']].append(i)
        
img_id = 4883 # test_id 시작주소
anno_id = 23144 # test_annotations 시작주소

for test_img in test['images']:
    
    new_annotations = []
    
    for annotations in preds[test_img['id']]:
        if annotations['score'] > score_thres: # score_thres

            x_min,y_min,x_max,y_max = annotations['bbox']
            x_min = round(x_min,1)
            y_min = round(y_min,1)
            x_max = round(x_max,1)
            y_max = round(y_max,1)
            new = {
                'image_id' : img_id,
                'category_id': annotations['category_id'],
                'area': x_max*y_max,
                'bbox':[x_min,y_min,x_max,y_max],
                'is_crowd': 0,
                'id': anno_id
            }
            new_annotations.append(new)
            anno_id+=1
    
    if len(new_annotations):
        test['id'] = img_id
        train['images'].append(test_img)
        train['annotations'].extend(new_annotations)
        img_id+=1

with open ('pseudo.json','w') as f:
    json.dump(train,f)
    


